In [ ]:
!pip install librosa

In [ ]:
!pip install audio_metadata

In [1]:
import librosa
import pandas as pd
import numpy as np
import os
import sys

from sklearn.model_selection import train_test_split

from tqdm import tqdm
from tinytag import TinyTag

In [2]:
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [3]:
tqdm.pandas()

# Load Data

In [4]:
DATA_DIR = 'data'

In [5]:
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')

In [6]:
value_map = {target: i  for i, target in enumerate(train_df.Target.unique())}
value_map_inv = {i: target  for i, target in enumerate(train_df.Target.unique())}

In [7]:
y_train = train_df.Target.apply(lambda t: value_map.get(t))

In [8]:
train_df.loc[:, 'Path'] = train_df.Filename.apply(lambda f: os.path.join(DATA_DIR, 'train', f))
test_df.loc[:, 'Path'] = test_df.Filename.apply(lambda f: os.path.join(DATA_DIR, 'test', f))

In [9]:
train_index, valid_index = train_test_split(train_df.index, test_size=.2)

# Text Features

In [10]:
import audio_metadata

In [11]:
train_tags = train_df.iloc[train_index].Path.progress_apply(lambda p: audio_metadata.load(p).tags)
train_titles = [tag.get('title', [''])[0] for tag in train_tags]
train_genres = [tag.get('genre', [''])[0] for tag in train_tags]

100%|██████████| 1498/1498 [00:01<00:00, 1168.48it/s]


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_title = TfidfVectorizer().fit(train_titles)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_genre = TfidfVectorizer(use_idf=False).fit(train_genres)

In [14]:
train_tfidf_title = tfidf_title.transform(train_titles).toarray()
train_tfidf_genre = tfidf_genre.transform(train_genres).toarray()

In [15]:
from sklearn.naive_bayes import MultinomialNB

In [16]:
nb_title = MultinomialNB().fit(train_tfidf_title, y_train[train_index])
nb_genre = MultinomialNB().fit(train_tfidf_genre, y_train[train_index])

In [17]:
nb_pred_train_title = nb_title.predict(train_tfidf_title)
nb_pred_train_genre = nb_genre.predict(train_tfidf_genre)

In [18]:
valid_tags = train_df.iloc[valid_index].Path.progress_apply(lambda p: audio_metadata.load(p).tags)
valid_titles = [tag.get('title', [''])[0] for tag in valid_tags]
valid_genres = [tag.get('genre', [''])[0] for tag in valid_tags]

100%|██████████| 375/375 [00:00<00:00, 1212.86it/s]


In [19]:
valid_tfidf_title = tfidf_title.transform(valid_titles).toarray()
valid_tfidf_genre = tfidf_genre.transform(valid_genres).toarray()

In [20]:
nb_pred_valid_title = nb_title.predict(valid_tfidf_title)
nb_pred_valid_genre = nb_genre.predict(valid_tfidf_genre)

In [21]:
print('Title Naive Bayes')
print(classification_report(y_train[valid_index], nb_pred_valid_title))

print('Genre Naive Bayes')
print(classification_report(y_train[valid_index], nb_pred_valid_genre))

Title Naive Bayes


NameError: name 'classification_report' is not defined

In [24]:
test_tags = test_df.Path.progress_apply(lambda p: audio_metadata.load(p).tags)
test_titles = [tag.get('title', [''])[0] for tag in test_tags]
test_genres = [tag.get('genre', [''])[0] for tag in test_tags]
test_tfidf_title = tfidf_title.transform(test_titles).toarray()
test_tfidf_genre = tfidf_genre.transform(test_genres).toarray()

100%|██████████| 253/253 [00:00<00:00, 3719.55it/s]


In [25]:
nb_pred_test_title = nb_title.predict(test_tfidf_title)
nb_pred_test_genre = nb_genre.predict(test_tfidf_genre)

# Audio Features

In [26]:
streams = train_df.Path.progress_apply(lambda f: librosa.load(f)[0])

100%|██████████| 1873/1873 [12:25<00:00,  2.51it/s]


In [27]:
train_tonnetz = streams.progress_apply(librosa.feature.tonnetz)

100%|██████████| 1873/1873 [05:33<00:00,  5.62it/s]


In [28]:
train_tonnetz_mean = train_tonnetz.apply(lambda t: np.mean(t, axis=1))

In [29]:
train_mfcc = streams.progress_apply(librosa.feature.mfcc)

100%|██████████| 1873/1873 [00:20<00:00, 92.41it/s]


In [30]:
train_mfcc_mean = train_mfcc.apply(lambda m: np.mean(m, axis=1))
train_mfcc_mean_ = np.array(train_mfcc_mean.tolist())

In [31]:
train_tempo = streams.progress_apply(librosa.beat.tempo)
train_tempo_ = np.array(train_tempo.tolist())

100%|██████████| 1873/1873 [00:48<00:00, 38.37it/s]


In [32]:
X = np.array(train_tonnetz_mean.tolist())
X = np.hstack((X, train_tempo_))
X = np.hstack((X, train_mfcc_mean_))

In [33]:
X_train, X_valid = X[train_index], X[valid_index]
y_train_, y_valid = y_train[train_index], y_train[valid_index]

In [ ]:
# X_train = np.hstack((X_train, nb_pred_train_title.reshape(-1, 1)))
# X_train = np.hstack((X_train, nb_pred_train_genre.reshape(-1, 1)))

# X_valid = np.hstack((X_valid, nb_pred_valid_title.reshape(-1, 1)))
# X_valid = np.hstack((X_valid, nb_pred_valid_genre.reshape(-1, 1)))

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [35]:
rt = RandomForestClassifier().fit(X_train, y_train_)

In [36]:
y_pred = rt.predict(X_valid)
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.74      0.76       141
           1       0.85      0.87      0.86       234

    accuracy                           0.82       375
   macro avg       0.81      0.81      0.81       375
weighted avg       0.82      0.82      0.82       375



In [37]:
test_streams = test_df.Path.progress_apply(lambda f: librosa.load(f)[0])

100%|██████████| 253/253 [01:39<00:00,  2.53it/s]


In [38]:
test_tonnetz = test_streams.progress_apply(librosa.feature.tonnetz)

100%|██████████| 253/253 [00:44<00:00,  5.74it/s]


In [39]:
test_tempo = test_streams.progress_apply(librosa.beat.tempo)

100%|██████████| 253/253 [00:06<00:00, 41.93it/s]


In [40]:
test_mfcc = test_streams.progress_apply(librosa.feature.mfcc)

100%|██████████| 253/253 [00:02<00:00, 99.95it/s] 


In [42]:
test_tonnetz_mean = test_tonnetz.apply(lambda t: np.mean(t, axis=1))
test_mfcc_mean = test_mfcc.apply(lambda t: np.mean(t, axis=1))

X_test = np.array(test_tonnetz_mean.tolist())
X_test = np.hstack((X_test, np.array(test_tempo.tolist())))
X_test = np.hstack((X_test, np.array(test_mfcc_mean.tolist())))

# X_test = np.hstack((X_test, nb_pred_test_title.reshape(-1, 1)))
# X_test = np.hstack((X_test, nb_pred_test_genre.reshape(-1, 1)))

In [43]:
y_pred = rt.predict(X_test)

In [44]:
test_df.loc[:, 'Prediction'] = y_pred
test_df.loc[:, 'Title'] = test_titles

In [61]:
y_pred_by_title = (test_df.groupby('Title').mean().Prediction > .75).astype(int)

In [62]:
y_pred_ = test_df.Title.apply(lambda t: y_pred_by_title.loc[t])

# Submission

In [63]:
pd.DataFrame({
    'Filename': test_df.Filename,
    'Target': [value_map_inv[y] for y in y_pred_],
}).to_csv('Submission-13.csv', index=False)